# Imports and Environment Setup



In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Image Preprocessing

- Defines functions for loading and preprocessing images to be compatible with the EfficientNet model input requirements.

In [19]:
# Function to load and prepare the image
def load_and_prepare_image(img_path, img_size=(224, 224)):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.efficientnet.preprocess_input(img_array_expanded_dims)


# Prediction Decoding

- Implements a function to decode model predictions into readable class names with their associated confidence scores, using the dataset directory to dynamically list class names.

In [20]:
# Function to decode predictions
def decode_predictions(predictions, top=3, dataset_path='./dataset'):
    # Dynamically list the class names based on folder names in the dataset_path
    class_names = sorted([dI for dI in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path,dI))])
    results = []
    for pred in predictions.numpy():
        top_indices = pred.argsort()[-top:][::-1]
        result = [(class_names[i], pred[i]) for i in top_indices]
        results.append(result)
    return results

# Model Loading and Inference Setup

- Loads the trained model and prepares it for making predictions.

In [21]:
# Load the trained model
model_path = './models/doodle_recognition'
loaded_model = tf.saved_model.load(model_path)
infer = loaded_model.signatures["serving_default"]

# Making Predictions on a Set of Images

- Processes a list of image paths, prepares each image, performs inference, and prints the top predictions for each image in the first set.

In [22]:
# List of images to predict
image_paths = ['./img/img1.png', './img/img2.png', './img/img3.png']

In [24]:
for img_path in image_paths:
    prepared_image = load_and_prepare_image(img_path)
    pred_logits = infer(tf.constant(prepared_image))['dense']
    top_predictions = decode_predictions(pred_logits, top=3)
    print(f"Predictions for {img_path}: {top_predictions}")

Predictions for ./img/img1.png: [[('dragon', 0.998961), ('mosquito', 0.00026180528), ('tiger', 0.00016646227)]]
Predictions for ./img/img2.png: [[('brain', 0.4601192), ('bush', 0.2767019), ('broccoli', 0.20393847)]]
Predictions for ./img/img3.png: [[('broccoli', 0.9135261), ('tree', 0.07400754), ('bear', 0.012262195)]]


- Repeats the prediction process for a second set of image paths, showcasing the model's ability to generalize across different images.

In [25]:
# List of images to predict
image_paths = ['./img/img4.png', './img/img5.png', './img/img6.png']

In [26]:
for img_path in image_paths:
    prepared_image = load_and_prepare_image(img_path)
    pred_logits = infer(tf.constant(prepared_image))['dense']
    top_predictions = decode_predictions(pred_logits, top=3)
    print(f"Predictions for {img_path}: {top_predictions}")

Predictions for ./img/img4.png: [[('donut', 0.9994673), ('ear', 0.00015448373), ('flying saucer', 0.0001226513)]]
Predictions for ./img/img5.png: [[('cruise ship', 0.902857), ('speedboat', 0.09713914), ('submarine', 3.038961e-06)]]
Predictions for ./img/img6.png: [[('house', 0.99989223), ('barn', 5.0330505e-05), ('lantern', 2.9691857e-05)]]
